# 添加消息历史记录

RunnableWithMessageHistory 允许我们将消息历史记录添加到某些类型的链中。它包装另一个 Runnable 并管理它的聊天消息历史记录。

具体来说，它可用于任何将以下之一作为输入的 Runnable

- 一个字典，其键采用 BaseMessage 序列
- 一个字典，其键将最新消息作为 BaseMessage 的字符串或序列，以及一个单独的键，将历史消息作为字符串或序列

并作为输出之一返回

- 可以视为 AIMessage 内容的字符串
- 一个字典，其键包含 BaseMessage 序列

让我们看一些示例，看看它是如何工作的。首先我们构造一个可运行的程序（这里接受一个字典作为输入并返回一条消息作为输出）：

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOACL_API_KEY")
api_base = os.getenv("LOACL_API_BASE")

model = ChatOpenAI(api_key=api_key, base_url=api_base, temperature=0.3)


prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '你是一个擅长{ability}的专家， 用20个字或更少的字数进行简洁概要的回复。'),
        MessagesPlaceholder(variable_name='history'),
        ('user', '{input}'),
    ]
)

runnable = prompt | model


为了管理消息历史记录，我们需要： 
1. 这个可运行程序； 
2. 返回 BaseChatMessageHistory 实例的可调用函数。

查看内存集成页面，了解使用 Redis 和其他提供程序实现聊天消息历史记录。在这里，我们演示如何使用内存中 ChatMessageHistory 以及使用 RedisChatMessageHistory 进行更持久的存储。

## In-memory 内存中

下面我们展示了一个简单的示例，其中聊天历史记录位于内存中，在本例中通过全局 Python 字典。

我们构造一个可调用的 get_session_history ，它引用此字典以返回 ChatMessageHistory 的实例。可以通过在运行时将配置传递给 RunnableWithMessageHistory 来指定可调用的参数。默认情况下，配置参数应为单个字符串 session_id 。这可以通过 history_factory_config kwarg 进行调整。

使用单参数默认值：

In [2]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(runnable, get_session_history=get_session_history, input_messages_key='input', history_messages_key='history')

请注意，我们指定了 input_messages_key （被视为最新输入消息的键）和 history_messages_key （添加历史消息的键）。

当调用这个新的可运行时，我们通过配置参数指定相应的聊天历史记录：

In [3]:
with_message_history.invoke(
    {"ability": "数学", "input": "余弦是什么意思？"},
    config={"configurable": {"session_id": "def234"}},
)

AIMessage(content='余弦是三角函数之一，表示直角三角形中，邻边与斜边之比。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 56, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-58d60d1c-10de-4d2b-98f0-4d73b8730380-0', usage_metadata={'input_tokens': 56, 'output_tokens': 31, 'total_tokens': 87, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
with_message_history.invoke(
    {"ability": "math", "input": "能否再说一遍？"},
    config={"configurable": {"session_id": "def234"}},
)

AIMessage(content='余弦是三角函数，用于表示直角三角形中，邻边与斜边之比。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 101, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3cc1f619-4b05-405f-ba96-6500b11d06b2-0', usage_metadata={'input_tokens': 101, 'output_tokens': 31, 'total_tokens': 132, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
with_message_history.invoke(
    {"ability": "math", "input": "能否再说一遍？"},
    config={"configurable": {"session_id": "ABC123"}},
)

AIMessage(content='擅长math，简洁概要回答问题。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 53, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9fc7a42b-0d74-4a4c-bdbd-d43dccd67bad-0', usage_metadata={'input_tokens': 53, 'output_tokens': 18, 'total_tokens': 71, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

我们可以通过将 ConfigurableFieldSpec 对象列表传递给 history_factory_config 参数来自定义用于跟踪消息历史记录的配置参数。下面，我们使用两个参数： user_id 和 conversation_id 。

In [6]:
from langchain_core.runnables import ConfigurableFieldSpec


store = {}

def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]

with_message_history = RunnableWithMessageHistory(
    runnable, 
    get_session_history=get_session_history, 
    input_messages_key='input',
    history_messages_key='history',
    history_factory_config=[
        ConfigurableFieldSpec(
            id='user_id', 
            name='User ID', 
            annotation=str,
            description='用户的唯一标识符',
            default='',
            is_shared=True
        ), 
        ConfigurableFieldSpec(
            id='conversation_id', 
            name='Conversation ID', 
            annotation=str,
            description='对话的唯一标识符',
            default='',
            is_shared=True
        )
    ]
)


In [7]:
with_message_history.invoke(
    {"ability": "数学", "input": "余弦是什么意思？"},
    config={"configurable": {"user_id": "kkutys", "conversation_id": "kkutys_conv1"}},
)


AIMessage(content='余弦是三角函数中的一种，表示直角三角形中，邻边与斜边的比值。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 56, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-486cc896-1ff1-4f32-a51c-a12b571d9fb1-0', usage_metadata={'input_tokens': 56, 'output_tokens': 34, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 具有不同签名的可运行实例的示例

上面的可运行程序接受一个字典作为输入并返回一个 BaseMessage。下面我们展示了一些替代方案。

In [8]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": model})

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history=get_session_history, 
    output_messages_key='output_message'
)

with_message_history.invoke(
    [HumanMessage(content="什么是二元一次方程？")],
    config={"configurable":{"session_id":"baz"}}
)

{'output_message': AIMessage(content='二元一次方程是指含有两个未知数的一次方程。一般形式为ax + by = c，其中a、b、c为已知数，x、y为未知数。解二元一次方程通常需要找到x和y的值，使得方程成立。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 18, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5f5f2330-ed0f-4379-b217-4836f10875dd-0', usage_metadata={'input_tokens': 18, 'output_tokens': 78, 'total_tokens': 96, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}

In [9]:
with_message_history.invoke(
    [HumanMessage(content="什么是未知数？")],
    config={"configurable":{"session_id":"baz"}}
)

{'output_message': AIMessage(content='未知数是在代数方程中代表未知数量的符号或变量。未知数通常用字母表示，如x、y、z等。在解方程或求解问题时，未知数的值是需要确定的，通过代数运算或方程求解可以得到未知数的具体值。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 111, 'total_tokens': 194, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3fc27af3-a82c-4d45-a24b-33de51264a9f-0', usage_metadata={'input_tokens': 111, 'output_tokens': 83, 'total_tokens': 194, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}

## 持久存储

在许多情况下，最好保留对话历史记录。 RunnableWithMessageHistory 不知道 get_session_history 可调用对象如何检索其聊天消息历史记录。有关使用本地文件系统的示例，请参阅此处。下面我们演示如何使用 Redis。查看内存集成页面，了解使用其他提供程序实现聊天消息历史记录。

如果尚未安装 Redis，我们需要安装它：

pip install --upgrade --quiet redis

如果我们没有可连接的现有 Redis 部署，请启动本地 Redis Stack 服务器：

docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest

REDIS_URL = "redis://localhost:6379/0"

更新消息历史记录实现只需要我们定义一个新的可调用对象，这次返回 RedisChatMessageHistory 的实例：


In [10]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


REDIS_URL = "redis://localhost:6379/0"

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    return RedisChatMessageHistory(url=REDIS_URL, session_id=session_id)

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history=get_session_history,
    input_messages_key='input',
    history_messages_key='history'
)

with_message_history.invoke(
    {"ability": "数学", "input": "余弦是什么意思？"},
    config={"configurable":{"session_id":"def234"}}
)

AIMessage(content='余弦是三角函数中的一种，表示直角三角形中的邻边与斜边的比值。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 56, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-89a6c425-005e-4c03-b3aa-76a8d9159483-0', usage_metadata={'input_tokens': 56, 'output_tokens': 34, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 50, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-66653df7-2837-4592-9137-e383a751a910-0', usage_metadata={'input_tokens': 50, 'output_tokens': 27, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
with_message_history.invoke(
    {"ability": "math", "input": "What's its inverse"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content='The inverse of cosine is called arccosine or cos^-1, and it returns the angle whose cosine is a given value.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 88, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-98dcc620-08ad-4296-8925-8c01ca3d9c43-0', usage_metadata={'input_tokens': 88, 'output_tokens': 28, 'total_tokens': 116, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})